## Deugging function of unity-based virtual reality rig

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from os.path import sep, isfile, exists
from os import mkdir, makedirs

In [2]:
from unityvr.preproc import logproc
from unityvr.viz import viz, utils

#### Construct data object from log file

In [43]:
#TODO: Add logging of 
# - frame rate
# - ball radius

In [44]:
dirName = "../sample/framerateTest/"
fileName = "Log_2021-01-14_17-20-18.json"

ftlog = 'fictrac-20210121_202559.dat'

setFramerate = 120# 144

In [ ]:
uvrTest = logproc.constructUnityVRexperiment(dirName,fileName)
uvrTest.printMetadata()

### Check frame rate
#### Basic visualizations

In [ ]:
framesDf = uvrTest.nidDf[['frame','time','dt']].drop_duplicates().reset_index(level=0)[['frame','time','dt']]

In [ ]:
fig, ax = plt.subplots(2,1, figsize=(10,6))
ax[0].plot(framesDf.time, 1/framesDf.dt, 'k')
ax[1].plot(framesDf.frame, 1/framesDf.dt, 'k')
for i in range(2):ax[i].axhline(setFramerate,0,1,color='r', linewidth=0.7)
utils.addlabs(ax, ['time [s]','frame'], ['frame rate [1/s]','frame rate [1/s]'])
fig.tight_layout()

utils.makemydir(sep.join([dirName,'plots']))
fig.savefig(sep.join([dirName,'plots','framerate_'+fileName.split('.')[0]+'.pdf']))

In [ ]:
fig, ax = plt.subplots(2,1, figsize=(10,6))
wind = 200
for i, ts in enumerate([0,5000]):
    ax[i].plot(range(wind),uvrTest.nidDf.pdsig[ts:ts+wind],'-',color='grey',linewidth=0.5)
    
    even = uvrTest.nidDf[ts:ts+wind][uvrTest.nidDf['frame'][ts:ts+wind]%2==1]
    ax[i].plot(np.arange(wind)[uvrTest.nidDf['frame'][ts:ts+wind].values%2==1], even.pdsig, 'b.', label='even frame')
    
    odd = uvrTest.nidDf[ts:ts+wind][uvrTest.nidDf['frame'][ts:ts+wind]%2==0]
    ax[i].plot(np.arange(wind)[uvrTest.nidDf['frame'][ts:ts+wind].values%2==0], odd.pdsig, 'c.', label=
              'odd frame')
    
utils.addlabs(ax, ['sample','sample'], ['photo diode signal','photo diode signal'])
ax[0].legend()
ax[0].set_title('Target frame rate: {} Hz ({}  s)'.format(setFramerate, round(1/setFramerate, 4)))
fig.tight_layout()

utils.makemydir(sep.join([dirName,'plots']))
fig.savefig(sep.join([dirName,'plots','photodiodeSig_evenframe_'+fileName.split('.')[0]+'.pdf']))

Visualization of delay

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(10,4))
for f in range(1000,1005):
    sig = uvrTest.nidDf.query('frame == '+str(float(f)))
    ax.plot(range(len(sig.pdsig)), sig.pdsig, '.-',  label='frame{}'.format(f))
ax.legend()

#### Quantification of frame rate variation

In [ ]:
fig, ax = plt.subplots(4,1, figsize=(10,10))
ts = 1000
wind = 300
ax[0].plot(uvrTest.nidDf.timeinterp[ts:ts+wind], uvrTest.nidDf.pdsig[ts:ts+wind], 'k')
ax[0].plot(uvrTest.nidDf.timeinterp[ts:ts+wind], uvrTest.nidDf.pdFilt[ts:ts+wind], 'c')
ax[0].plot(uvrTest.nidDf.timeinterp[ts:ts+wind], uvrTest.nidDf.pdThresh[ts:ts+wind], '--',color='orange')

pdChange = np.hstack((0,np.diff(uvrTest.nidDf.pdThresh)))
pdChangeT = uvrTest.nidDf.timeinterp.values[abs( pdChange )>0]

ax[1].plot(uvrTest.nidDf.timeinterp, uvrTest.nidDf.pdsig, 'k')

ax[2].plot(pdChangeT[1:], 1/np.diff(pdChangeT), '.', color='grey')
ax[2].axhline(setFramerate,0,1,color='r', linewidth=0.7)

ax[3].hist(1/np.diff(pdChangeT), bins=50, range=(0, 300), color='grey')
ax[3].set_xlim(0,250)
ax[3].axvline(setFramerate,0,1,color='r', linewidth=0.7)

ax[0].set_title('Target frame rate: {} Hz ({}  s)'.format(setFramerate, round(1/setFramerate, 4)))

fig.tight_layout()
utils.addlabs(ax, ['time [s]','time [s]','time [s]','photodiode-based frame rate [1/s]'],
    ['photodiode signal [V]','photodiode signal [V]','photodiode-based\nframe rate [1/s]','count'])

utils.makemydir(sep.join([dirName,'plots']))
fig.savefig(sep.join([dirName,'plots','photdiodeSignal_framerate_'+fileName.split('.')[0]+'.pdf']))

#### Relate unity-based and photodiode signal-based frame rate measures

In [ ]:
pdChangedt = uvrTest.nidDf.dt.values[abs( pdChange )>0]

fig, axs = plt.subplots(1,1,figsize=(4,4))

axs.plot(np.roll(pdChangedt,0), np.hstack((0,np.diff(pdChangeT))), '.', color='grey', alpha=0.3)
#axs.plot(np.roll(pdChangedt,10), np.hstack((0,np.diff(pdChangeT))), 'r.', alpha=0.5)
axs.set_xlim(0,0.05)
axs.set_ylim(0,0.05)
axs.set_xlabel('Unity dt [s]')
axs.set_ylabel('Photodiode dt [s]')
axs.axvline(1/setFramerate,0,1,color='r', linewidth=0.7)
axs.axhline(1/setFramerate,0,1,color='r', linewidth=0.7)
axs.set_aspect('equal')
ax[0].set_title('Target frame rate: {} Hz ({}  s)'.format(setFramerate, round(1/setFramerate, 4)))

utils.makemydir(sep.join([dirName,'plots']))
fig.savefig(sep.join([dirName,'plots','unityVsPhotodiode_framerate_'+fileName.split('.')[0]+'.pdf']))

### Compare unityVR trajectory with fictrac trajectory
#### Compare frame rates

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(10,4))
ax.plot(uvrTest.posDf.frame, 1/uvrTest.posDf.dt, color='grey', label="unity frame rate")
ax.plot(uvrTest.ftDf.frame,np.hstack([ 0, 1/np.diff(uvrTest.ftDf.ficTracTReadMs/1000.) ]), 'c-', label='fictrac frame rate')
print('Unity log frame rate: {}'.format(1/uvrTest.posDf.dt.mean()))
print('Fictrac log frame rate: {}'.format(1/np.diff(uvrTest.ftDf.ficTracTReadMs/1000.).mean()))
ax.set_xlim(500,1000)
ax.set_ylim(50,250)
ax.axhline(setFramerate,0,1,color='r', linewidth=0.7, label='set unity frame rate')
ax.axhline(130,0,1,color='b', linewidth=0.7, label="approximate fictrac camera fram rate")
ax.legend()

#### Compare to fictrac data log

In [ ]:
ftdat = pd.read_csv(sep.join([dirName, ftlog]), header=None)

In [ ]:
ftdat.head()

In [ ]:
fig, axs = plt.subplots(3,1, figsize=(10,10))

axs[0].plot(uvrTest.posDf.frame,uvrTest.posDf.dt)
axs[1].plot(uvrTest.ftDf.frame, np.gradient(uvrTest.ftDf.ficTracTReadMs/1000.))

ftdt = ftdat[23].values/1000.
axs[2].plot(ftdat[0].values[1:-1], ftdt[1:-1])

#### Vizualize trajectory and object positions

In [ ]:
uvrTest.ftDf.keys()

In [ ]:
ballr = 40
convf = ballr #ballr*np.pi/180
#plt.plot(posDf['x'], posDf['y'],color='grey',alpha=0.5)
#plt.scatter(posDf['x'], posDf['y'],s=7,c=posDf['time'],cmap='viridis')

# See Seelig 2010 for reference on equations (https://www.ncbi.nlm.nih.gov/pmc/articles/PMC2945246/)
fta = np.cumsum(uvrTest.ftDf['dz'])
fty = np.cumsum(convf*uvrTest.ftDf['dy']*np.cos(fta) - convf*uvrTest.ftDf['dx']*np.sin(fta))
ftx = np.cumsum(convf*uvrTest.ftDf['dy']*np.sin(fta) + convf*uvrTest.ftDf['dx']*np.cos(fta))
plt.plot(ftx[0:3000], fty[0:3000],color='grey',alpha=0.5)
plt.scatter(ftx[0:3000], fty[0:3000],s=7,c=uvrTest.ftDf['frame'][0:3000],cmap='viridis')

In [ ]:
plt.plot(uvrTest.posDf['x'], uvrTest.posDf['y'],color='grey',alpha=0.5)
plt.scatter(uvrTest.posDf['x'], uvrTest.posDf['y'],s=7,c=uvrTest.posDf['time'],cmap='viridis')

In [ ]:
from importlib import reload  
reload(viz)

In [ ]:
fig = viz.plotVRpathWithObjects(uvrTest, limx=[-400,400], limy=[-300,300],myfigsize=(7,7))